In [1]:
import numpy as np
import pandas as pd
import pyspark
from pyspark.sql import SparkSession

# Load in

In [2]:
conf = pyspark.SparkConf().setAll([
    ('spark.master', 'local[1]'),
    ('spark.app.name', 'App Name')])
spark = SparkSession.builder.config(conf=conf).getOrCreate()

spark.version

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/04/16 18:20:05 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


'3.3.1'

In [ ]:
spark = SparkSession.builder.master("local[1]") \
          .appName("SparkByExamples.com") \
          .getOrCreate()

In [4]:
df = spark.sparkContext.textFile('../../../../cephfs/cleanedData/2022-02_sample.csv') #.toDF()
# df = spark.read.csv("../../cephfs/cleanedData/2022-02_sample.csv")
# df.printSchema()
df.collect()

# int, datetime, string, float, int

['PatientId,GlucoseDisplayTime,GlucoseDisplayTimeRaw,Value,dayCSV',
 '2189,2022-02-01 01:35:54.162,2022-02-01T01:35:54.162-06:00,0,1',
 '2189,2022-01-31 18:10:49.212,2022-01-31T18:10:49.212-06:00,0,1',
 '2189,2022-01-31 22:20:51.624,2022-01-31T22:20:51.624-06:00,0,1',
 '2189,2022-02-01 08:00:59.366,2022-02-01T08:00:59.366-06:00,0,1',
 '2189,2022-02-01 15:20:57.416,2022-02-01T15:20:57.416-06:00,0,1',
 '2189,2022-02-01 03:00:55.234,2022-02-01T03:00:55.234-06:00,0,1',
 '2189,2022-02-01 14:40:54.135,2022-02-01T14:40:54.135-06:00,0,1',
 '2189,2022-01-31 17:45:49.146,2022-01-31T17:45:49.146-06:00,0,1',
 '2189,2022-02-01 07:15:53.908,2022-02-01T07:15:53.908-06:00,0,1',
 '2189,2022-02-01 13:45:53.871,2022-02-01T13:45:53.871-06:00,0,1',
 '2189,2022-02-01 10:30:56.756,2022-02-01T10:30:56.756-06:00,0,1',
 '2189,2022-02-01 10:40:52.621,2022-02-01T10:40:52.621-06:00,0,1',
 '2189,2022-02-01 12:20:54.263,2022-02-01T12:20:54.263-06:00,0,1',
 '2189,2022-02-01 08:30:54.109,2022-02-01T08:30:54.109-06:00,

In [ ]:
df.toDF(

In [19]:
'''use this'''
from pyspark.sql.types import StructType,StructField, StringType, IntegerType, DoubleType, TimestampType
dfSchema = StructType([       
    StructField('PatientId', IntegerType(), True),
    StructField('GlucoseDisplayTime', TimestampType(), True),
    StructField('GlucoseDisplayTimeRaw', StringType(), True),
    StructField('Value', DoubleType(), True),
    StructField('dayCSV', IntegerType(), True)
])
# spark.createDataFrame(df, schema = dfSchema)

'''read in the csv'''
df1 = spark.read\
    .option("mode", "DROPMALFORMED") \
    .option("delimiter",",") \
    .schema(dfSchema) \
    .csv('../../../../cephfs/cleanedData/2022-02_sample.csv')

df1.select("*").show()

In [7]:
df1 = spark.createDataFrame(df, schema = dfSchema)
df1.printSchema()
# df1.head(5)

root
 |-- PatientId: string (nullable = true)
 |-- GlucoseDisplayTime: string (nullable = true)
 |-- GlucoseDisplayTimeRaw: string (nullable = true)
 |-- Value: string (nullable = true)
 |-- dayCSV: string (nullable = true)



In [5]:
headers= ['PatientId','GlucoseDisplayTime','GlucoseDisplayTimeRaw','Value','dayCSV']
# df.toDF(headers)
spark.createDataFrame(df, schema=headers).collect()

TypeError: Can not infer schema for type: <class 'str'>

# Prelim cleanup before filling

### replace 0s with NaN to save a two steps down the line

In [ ]:
# lines.na.fill(value=0,subset=["population"]).show()
lines.withColumn("Value", \
       when(col("Value")=="0", None) \
          .otherwise(col("Value"))) \
  .show()